In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
import requests

In [32]:
json={
    'participant_id': 0s
}
req = requests.post("http://ec2-52-24-126-225.us-west-2.compute.amazonaws.com:81/iris/retrieve-participant-preferences", json=json)
queries =pd.DataFrame(req.json()["data"])
queries

In [33]:
req.ok

True

,cluster_id,confidence,file_path,gcs_arrival_time,level_of_autonomy,mission_id,operators_response,preferred_level_of_autonomy,query_id,robot_id,true_response,type,user_id
0,-1,0.94,test.png,3,0,0,2,0,643,3,0,1,0
1,0,0.50,fart.png,2,0,0,3,1,679,0,2,3,0
2,-1,0.50,fart.png,2,1,0,2,0,680,0,0,0,0
3,-1,0.60,fart.png,0,0,1,3,0,700,0,1,0,0
4,-1,0.90,fart.png,1,0,1,0,0,701,0,1,3,0
5,-1,0.20,fart.png,1,1,1,3,0,702,0,0,3,0
6,-1,0.70,fart.png,3,0,1,3,0,703,0,3,1,0
7,-1,0.30,fart.png,2,1,2,3,0,784,0,0,0,0
8,-1,0.10,fart.png,4,0,2,1,0,785,0,3,3,0
9,-1,0.80,fart.png,2,0,2,0,0,786,0,1,1,0


In [35]:
queries.loc[0,'SurveyName']="Debrief"
queries.loc[1,'SurveyName']="Order"
queries.loc[2,'SurveyName']=str(0)
queries.loc[3,'SurveyName']= "Last Edited By"
queries.loc[4,'SurveyName']="Python Script"
queries.loc[5,'SurveyName']="Who knows?"
'''
+ str(0),
        "Order",
        str(0),
        "Last Edited By",
        "Python Script", 
        "Last Edited",
        "Who knows?"]
'''
queries

,cluster_id,confidence,file_path,gcs_arrival_time,level_of_autonomy,mission_id,operators_response,preferred_level_of_autonomy,query_id,robot_id,true_response,type,user_id,SurveyName
0,-1,0.94,test.png,3,0,0,2,0,643,3,0,1,0,Debrief
1,0,0.50,fart.png,2,0,0,3,1,679,0,2,3,0,Order
2,-1,0.50,fart.png,2,1,0,2,0,680,0,0,0,0,0
3,-1,0.60,fart.png,0,0,1,3,0,700,0,1,0,0,Last Edited By
4,-1,0.90,fart.png,1,0,1,0,0,701,0,1,3,0,Python Script
5,-1,0.20,fart.png,1,1,1,3,0,702,0,0,3,0,Who knows?
6,-1,0.70,fart.png,3,0,1,3,0,703,0,3,1,0,NaN
7,-1,0.30,fart.png,2,1,2,3,0,784,0,0,0,0,NaN
8,-1,0.10,fart.png,4,0,2,1,0,785,0,3,3,0,NaN
9,-1,0.80,fart.png,2,0,2,0,0,786,0,1,1,0,NaN


In [5]:
class SurveyGenerator:
    def __init__(self):
        #json={
        #    'participant_id': 0
        #}
        #req = requests.post("http://ec2-52-24-126-225.us-west-2.compute.amazonaws.com:81/iris/retrieve-participant-preferences", json=json)
        
        self.clusters_sql = pd.DataFrame()
        self.clusters_sql['question_id']=[]
        self.clusters_sql['cluster_id']=[]
        self.clusters_sql['query_id']=[]
            #queries =pd.DataFrame(req.json()["data"])
        
        
        

    
    
    def generate_and_upload (self,queries,debrief = 0,transparency=False,random=False):
        self.debrief = debrief + 7
        self.survey_csv = pd.DataFrame()
        self.survey_column_name = "Survey Name"
        self.question_column_name = "Question"
        self.type_column_name = "Type"
        self.offered_answers_name = "Official Answers"
        
        
        
        self.survey_csv[self.survey_column_name]= []
        self.survey_csv[self.question_column_name]= []
        self.survey_csv[self.type_column_name]= []
        self.survey_csv[self.offered_answers_name]= []
        
        self.survey_csv.loc[0,'Survey Name']="Debrief " + str(self.debrief)
        self.survey_csv.loc[1,'Survey Name']="Order"
        self.survey_csv.loc[2,'Survey Name']=str(self.debrief)
        self.survey_csv.loc[3,'Survey Name']= "Last Edited By"
        self.survey_csv.loc[4,'Survey Name']="Python Script"
        self.survey_csv.loc[5,'Survey Name']="Who knows?"
        
        
        iURL = 'http://ec2-52-24-126-225.us-west-2.compute.amazonaws.com:81/sql-temp/insert'
        
        if not random:
            self.queries = self._select_random_queries(queries)
        else:
            self.queries = queries
        if transparency:
            self._high_transparency()
        else:
            self._low_transparency()
        
        
        self.survey_csv = self.survey_csv.replace(np.nan, '', regex=True)
        
        outname = 'debrief{0}.csv'.format(self.debrief)
        outdir = 'debrief_surveys'
        if not os.path.exists(outdir):
            os.mkdir(outdir)

        fullname = os.path.join(outdir, outname) 
        self.path = fullname

        import csv
        keys = (self.survey_csv.keys())
        with open(fullname, "wb") as outfile:
           writer = csv.writer(outfile, delimiter = ",")
           writer.writerow(keys)
           writer.writerows(zip(*[self.survey_csv[key] for key in keys]))
        #self.upload_survey_to_database(iURL)
        print(self.clusters_sql) 
        self._send_clusters()
       
    def _send_clusters (self):
        sql_template = 'UPDATE dbexperiment.clusters SET  question_id = {0} WHERE cluster_id = {1} AND query_id = {2};'
        iURL = 'http://ec2-52-24-126-225.us-west-2.compute.amazonaws.com:81/sql-temp/update'
        for index, cluster in self.clusters_sql.iterrows():
            print (cluster)
            query = sql_template.format(cluster['question_id'],cluster['cluster_id'],cluster['query_id'])
            self.send_query(query,iURL)
            
            
    def _select_random_queries(self,queries):
        new_queries =queries.sample(frac=1).copy
        new_queries = new_queries.im_self.drop_duplicates(subset=['cluster_id'], keep='last')
        new_queries = new_queries.reset_index()
        return new_queries

    def send_query(self,query,URL):
        iData = {
                    'sql': query,
                    'database': 'dbsurveys'
                }
        req = requests.post(url=URL, data=iData)
        if "200" in req:
            print(query)
        print(req)
        print(req.text)
        
    def upload_survey(self,path,iURL):
        print ("path = "+ path)
        data = pd.read_csv(path)
        print (data.iloc[0][0])
        print (data.iloc[2][0])
        survey_name = data.iloc[0][0]
        survey_id = int(data.iloc[2][0])
        data = data.drop(columns=[self.survey_column_name])
        data = data.dropna(axis=0, how='all')

        #will craete a new entry if it does not already exists
        survey_query =("INSERT INTO dbsurveys.survey (id, description)" + 
                        "SELECT * FROM (SELECT '{0}', '{1}') AS tmp " + 
                        "WHERE NOT EXISTS (" + 
                        "SELECT description FROM dbsurveys.survey " +
                        "WHERE description = '{1}') LIMIT 1;")
        survey_query = survey_query.format(survey_id,survey_name)

        question_query = "INSERT INTO dbsurveys.question (id, type, question_text) VALUE "
        offered_answer_query = "INSERT INTO dbsurveys.offered_answer (id, offered_answer_text) VALUE"
        survey_question_query = "INSERT INTO dbsurveys.survey_question (survey_id, question_id) VALUE "
        question_offered_answer_query =  "INSERT INTO dbsurveys.question_answer (question_id, offered_answer_id) VALUE "
        #offered_answer_query = "INSERT INTO dbsurveys.question d(id, offered_answer_text) values"
        #str = str.replace("'", "\\'")
        answers = []

        for i, row in data.iterrows():
            #if index == 1:
             #   break
            index = i + survey_id*100
            question_text = str(row[self.question_column_name]).replace("'", "\\'")
            #
            question_type = str(row[self.type_column_name]).replace("'", "\\'")
            offered_answer = ""
            if(not pd.isnull(row[self.offered_answers_name])):
                offered_answer = row[self.offered_answers_name]
            if pd.isnull(row[self.question_column_name]):
                break

            question_query += "('{0}', '{1}', '{2}')".format(index,question_type, question_text)
            if(pd.isnull(row[self.offered_answers_name])):
                offered_answer_text = ""
                offered_answer_query += "('{0}', '{1}')".format(index, offered_answer_text)
            else:
                
                offered_answer_text = str(row[self.offered_answers_name]).replace("'", "\\'")
                offered_answer_query += "('{0}','{1}')".format(index, offered_answer_text)
            survey_question_query += "('{0}','{1}')".format(survey_id,index)
            question_offered_answer_query += "('{0}','{1}','{2}')".format(survey_id,index,index)
            #if index > 5:
             #       break

            if i < len(data.index) - 1:
                question_query += ","
                offered_answer_query += ","
                survey_question_query += ","
                question_offered_answer_query += ","
          
           
        question_query += ";"
        offered_answer_query += ";"
        survey_question_query += ";"
        question_offered_answer_query += ";"
        
            #breaks post request for me
        query = survey_query+ question_query + offered_answer_query + survey_question_query + question_offered_answer_query    
        print (query)
        self.send_query(survey_query , iURL)
        self.send_query(question_query , iURL)
        self.send_query(offered_answer_query , iURL)
        self.send_query(survey_question_query , iURL)
        self.send_query(question_offered_answer_query , iURL)
        
            
    def _high_transparency(self):
        print ("in high transparency")
        for index, query in self.queries.iterrows():
            #q_id = 0
            print (index)
            self.clusters_sql.loc[index,"question_id"]=index
            self.clusters_sql.loc[index,"cluster_id"]=query["cluster_id"]
            self.clusters_sql.loc[index,"query_id"]=query["query_id"]
            self.survey_csv.loc[index,self.type_column_name]="Debrief"
            if query['level_of_autonomy'] is 0:
                self.survey_csv.loc[index,self.question_column_name]= (
                    'I asked Husky {0} to stop and wait '
                    'for your response to this {1} query with a {2}% confidence level. '
                    'How would you like me to handle similar queries next time?').format(
                        query['robot_id'],
                        query['type'],
                        query['confidence'])
                self.survey_csv.loc[index,self.offered_answers_name]=(
                    'Please ask a Husky to handle similar queries in the next mission|'
                    'Please ask a Husky to stop and wait until I answer similar queries in the next mission')
            else:
                self.survey_csv.loc[index,self.question_column_name]=(
                    'Husky [#} answered this {1} query, with a {2} confidence level'
                    'while you were tending to another query.'
                    'How would you like me to handle similar queries next time?').format(
                        query["robot_id"],
                        query['type'],
                        query['confidence'])
                    
                self.survey_csv.loc[index,self.offered_answers_name]=(
                    'Please ask a Husky to handle similar queries in the next mission|'
                    'Please ask a Husky to stop and wait until I answer similar queries in the next mission')
            #q_id+=1
    def _low_transparency(self):
        print ("in low transparency")
        for index, query in self.queries.iterrows():
            q_id = 0
            
            self.clusters_sql.loc[index,"question_id"]=index
            self.clusters_sql.loc[index,"cluster_id"]=query["cluster_id"]
            self.clusters_sql.loc[index,"query_id"]=query["query_id"]
            str(q_id)
            self.survey_csv.loc[index,self.type_column_name]="Debrief"
            if query['level_of_autonomy'] == 0:
                self.survey_csv.loc[index,self.question_column_name]= ('This {0} query was answered by you. '
                    'How would you like me to handle similar queries next time?').format(query["type"])
                self.survey_csv.loc[index,self.offered_answers_name]=('Please ask a Husky to handle similar queries in the next mission|'
                    'Please ask a Husky to stop and wait until I answer similar queries in the next mission')
            else:
                self.survey_csv.loc[index,self.question_column_name]= ('This {0} query was answered by Husky {1} '
                    'while you were attending to another query. How would you like me to handle similar queries next time?').format(
                    query["robot_id"],
                    query['type'])
                    
                self.survey_csv.loc[index,self.offered_answers_name]=('Please ask a Husky to handle similar queries in the next mission|' 
                    'Please ask a Husky to stop and wait until I answer similar queries in the next mission')
            q_id += 1
            
    def upload_survey_to_database(self,URL):
        self.upload_survey(self.path,iURL=URL)
    def upload_clusters_to_database(self,URL):
        survey_query =("INSERT INTO dbexperiment.clusters (query_id, cluster_id, question_id)" 
                        "VALUE ({0},{1},{2});")
        for cluster in self.clusters_sql.iterrows():
            query = survey_query.format(cluster["query_id",cluster["cluster_id"],cluster["question_id"]])
        
            upload_surveys.send_query(query,URL)

In [6]:
attempt1 = SurveyGenerator()

In [14]:
for i in range(5):
    json={
        'participant_id': 0
    }
    req = requests.post("http://ec2-52-24-126-225.us-west-2.compute.amazonaws.com:81/iris/retrieve-participant-preferences", json=json)
    queries =pd.DataFrame(req.json()["data"])
    queries
    attempt1.generate_and_upload(queries,debrief=i)

in low transparency
   question_id  cluster_id  query_id
0          0.0         0.0     679.0
1          1.0        -1.0     703.0
question_id      0.0
cluster_id       0.0
query_id       679.0
Name: 0, dtype: float64
<Response [200]>
{"response":true,"data":{"fieldCount":0,"affectedRows":1,"insertId":0,"serverStatus":2,"warningCount":0,"message":"(Rows matched: 1  Changed: 0  Warnings: 0","protocol41":true,"changedRows":0}}
question_id      1.0
cluster_id      -1.0
query_id       703.0
Name: 1, dtype: float64
<Response [200]>
{"response":true,"data":{"fieldCount":0,"affectedRows":1,"insertId":0,"serverStatus":2,"warningCount":0,"message":"(Rows matched: 1  Changed: 1  Warnings: 0","protocol41":true,"changedRows":1}}
in low transparency
   question_id  cluster_id  query_id
0          0.0         0.0     679.0
1          1.0        -1.0     703.0
question_id      0.0
cluster_id       0.0
query_id       679.0
Name: 0, dtype: float64
<Response [200]>
{"response":true,"data":{"fieldCount":